In [0]:
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import nltk
import pandas as pd
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def df_to_POS(df):
    l = df["_label"]
    df=df.drop("_label", axis = 1)
    
    col = df.columns
    A = []
    
    for i in range(len(col)):
        c = col[i]
        pp = nltk.pos_tag(nltk.word_tokenize(c))
        a = ""
        for k in range(len(pp)):
            a = a + "_" + pp[k][1]
        A.append(a)
    
    
    pos = pd.DataFrame()
    for p in np.unique(A):
    
        np.where(p == np.array(A))
        pos[p] = df[df.columns[np.where(p == np.array(A))]].sum(axis = 1)
    
    pos["_label"] = l
    return(pos)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [0]:
df1  = pd.read_pickle("/content/drive/My Drive/Dataset/1gram.pkl")
pos1 = df_to_POS(df1)
del(df1)

In [0]:
df2  = pd.read_pickle("/content/drive/My Drive/Dataset/2gram.pkl")
pos2 = df_to_POS(df2)
del(df2)

In [0]:
df3  = pd.read_pickle("/content/drive/My Drive/Dataset/3gram.pkl")
pos3 = df_to_POS(df3)
del(df3)

In [0]:
df45  = pd.read_pickle("/content/drive/My Drive/Dataset/45gram.pkl")
pos45 = df_to_POS(df45)
del(df45)

In [9]:
pos = pos1.merge(pos2, on = "_label").merge(pos3, on = "_label").merge(pos45, on = "_label")
pos.shape, pos1.shape, pos2.shape, pos3.shape, pos45.shape

((6406, 5591), (6407, 24), (6406, 307), (6406, 1174), (6406, 4089))

In [0]:
country = []
for i in pos['_label']:
    country.append(i.split("/")[5])
pos['country']=country

In [11]:
pos.head(3)

,_CC,_CD,_DT,_IN,_JJ,_JJR,_JJS,_MD,_NN,_NNS,_PRP,_PRP$,_RB,_RBR,_TO,_VB,_VBD,_VBG,_VBN,_WDT,_WP,_WP$,_WRB,_label,_CC_CC,_CC_CD,_CC_DT,_CC_EX,_CC_IN,_CC_JJ,_CC_JJR,_CC_JJS,_CC_MD,_CC_NN,_CC_NNS,_CC_PRP,_CC_PRP$,_CC_RB,_CC_RBR,_CC_TO,...,_WP_VBP_IN_NN_IN,_WP_VBP_IN_RB,_WP_VBP_IN_RB_DT,_WP_VBP_NNS_CC,_WP_VBP_NNS_CC_JJ,_WP_WRB_WRB_WRB,_WP_WRB_WRB_WRB_WP,_WRB_CC_VB_DT,_WRB_DT_JJ_NN_IN,_WRB_DT_NN_IN,_WRB_DT_NN_IN_NN,_WRB_DT_NN_NN,_WRB_DT_NN_NN_NN,_WRB_DT_NN_VB,_WRB_DT_NN_VB_RB,_WRB_EX_VB_DT,_WRB_JJ_NN_TO,_WRB_JJ_NN_TO_DT,_WRB_NN_IN_DT,_WRB_NN_NN_NN,_WRB_NN_NN_NN_NN,_WRB_NN_NN_NN_VB,_WRB_NN_NN_VB,_WRB_NN_NN_VB_JJ,_WRB_NN_TO_DT,_WRB_NN_TO_DT_JJ,_WRB_NN_VB_DT,_WRB_NN_VB_JJ,_WRB_NN_VB_JJ_NN,_WRB_NN_VB_NN,_WRB_PRP_VBD_TO,_WRB_PRP_VB_TO_NN,_WRB_TO_VB_DT,_WRB_TO_VB_DT_NN,_WRB_VB_DT_JJ,_WRB_VB_DT_NN,_WRB_VB_DT_NN_IN,_WRB_VB_JJ_JJ,_WRB_WRB_WRB_WP,country
0,252,20,628,965,406,13,8,105,2354,310,79,35,257,10,171,403,0,7,2,9,16,1,28,/home/marcin/Dane/Manjunath/USA/AdelsteinDietr...,0.0,0.0,12.0,0.0,8.0,5.0,0,0.0,4.0,35.0,10.0,1.0,7.0,7.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,USA
1,322,19,359,919,346,2,32,41,2706,460,171,171,254,29,167,374,0,0,1,11,14,0,56,/home/marcin/Dane/Manjunath/USA/AlNoamanyBorgh...,2.0,0.0,18.0,0.0,10.0,8.0,0,3.0,1.0,40.0,6.0,2.0,7.0,11.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,USA
2,243,59,511,849,416,0,20,58,2506,299,145,31,155,23,197,370,0,0,0,27,12,0,19,/home/marcin/Dane/Manjunath/USA/AllegranteJohn...,1.0,2.0,14.0,0.0,7.0,9.0,0,1.0,0.0,32.0,4.0,3.0,5.0,5.0,5.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,USA


In [0]:
pos.to_csv("zbiorek.csv")

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [0]:
X = pos.drop("_label", axis=1).drop("country", axis=1)
y = pos['country']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [0]:
clf = RandomForestClassifier(n_estimators=1000,random_state=0)

In [0]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [0]:
y_pred=clf.predict(X_test)

In [0]:
y_pred_train=clf.predict(X_train)

In [0]:
sum(y_pred==y_test)/len(y_test), sum(y_pred_train==y_train)/len(y_train)

(0.5993377483443708, 0.999767008387698)

In [0]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[294,   0,   0,   0,   0,   0,   4,   0,   0,   1,  15,   0],
       [ 16,  71,   0,   1,   0,   2,  11,  24,   3,   9,  25,   0],
       [ 12,   1,  37,   0,   0,   0,   7,   5,   2,  11,  32,   0],
       [  7,   4,   2,  31,   1,   0,  10,  40,   4,   9,  22,   0],
       [ 23,   0,   2,   0,  22,   3,  10,   1,   5,   6,  43,   0],
       [  7,   6,   0,   0,   0,  40,  35,  14,   3,   6,  16,   0],
       [  6,   1,   0,   0,   0,   2, 141,   3,   0,   5,  25,   0],
       [ 11,   0,   0,   0,   0,   5,   7, 149,   0,   2,  10,   0],
       [ 27,   5,   0,   0,   0,   0,  18,   5,  61,  10,  21,   0],
       [ 20,   1,   4,   0,   0,   4,   4,   5,   3, 124,  68,   0],
       [ 16,   1,   2,   0,   0,   0,   1,   2,   2,  20, 242,   1],
       [ 43,   0,   1,   0,   0,   0,   7,   6,   0,   0,  13,  55]])

In [0]:
np.unique(y_train)

array(['China', 'France', 'Germany', 'Italy', 'Japan', 'Poland', 'Russia',
       'Spain', 'Turkey', 'UK', 'USA', 'Vietnam'], dtype=object)

In [0]:
clf.feature_importances_

array([3.21709384e-03, 2.41425595e-03, 2.79310175e-03, ...,
       3.44607939e-04, 2.59433007e-07, 4.60375174e-07])

In [0]:
pd.DataFrame(clf.feature_importances_, X.columns).sort_values(0, ascending = False)[0:5]

,0
_JJ_JJ_NN_NN_IN,0.005600
_JJ_JJ_NN_NN,0.005013
_DT_VB,0.004086
_TO_VB_DT,0.003746
_DT_VBP,0.003723


In [0]:
pos.to_pickle("pos.pkl")

In [0]:
from google.colab import files
files.download('pos.pkl') 

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 38958, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil